Extract BTC information

In [6]:
import requests
import pandas as pd
from datetime import datetime, timedelta

In [7]:
def fetch_binance_data(symbol, interval, start_date, end_date=None):
    """
    Descarga datos históricos de Binance en intervalos definidos.
    
    Args:
        symbol (str): El par de trading, por ejemplo, "BTCUSDT".
        interval (str): Intervalo de tiempo (1m, 1h, 1d, etc.).
        start_date (str): Fecha de inicio en formato "YYYY-MM-DD".
        end_date (str): Fecha de fin en formato "YYYY-MM-DD". (Opcional)
        
    Returns:
        pd.DataFrame: DataFrame con los datos históricos.
    """
    base_url = "https://api.binance.com/api/v3/klines"
    start_time = int(datetime.strptime(start_date, "%Y-%m-%d").timestamp() * 1000)
    end_time = int(datetime.strptime(end_date, "%Y-%m-%d").timestamp() * 1000) if end_date else int(datetime.now().timestamp() * 1000)
    
    data = []
    while start_time < end_time:
        params = {
            "symbol": symbol,
            "interval": interval,
            "startTime": start_time,
            "endTime": min(start_time + 1000 * 60 * 60 * 24 * 30, end_time),  # Máximo 30 días por solicitud
            "limit": 1000  # Límite máximo por solicitud
        }
        response = requests.get(base_url, params=params)
        response_data = response.json()
        if len(response_data) == 0:
            break
        data.extend(response_data)
        start_time = response_data[-1][6] + 1  # Actualiza el tiempo de inicio
    
    # Crear DataFrame
    columns = [
        "Open Time", "Open", "High", "Low", "Close", "Volume", "Close Time",
        "Quote Asset Volume", "Number of Trades", "Taker Buy Base Volume",
        "Taker Buy Quote Volume", "Ignore"
    ]
    df = pd.DataFrame(data, columns=columns)
    
    # Convertir timestamps a fechas legibles
    df["Open Time"] = pd.to_datetime(df["Open Time"], unit="ms")
    df["Close Time"] = pd.to_datetime(df["Close Time"], unit="ms")
    
    return df


In [8]:
# Descargar datos de BTCUSDT desde el inicio de Binance
symbol = "BTCUSDT"
interval = "1d"  # Intervalo de tiempo (1d = 1 día)
start_date = "2024-01-01"  # Fecha de inicio para BTCUSDT en Binance
df = fetch_binance_data(symbol, interval, start_date)




In [9]:
df

,Open Time,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,Taker Buy Base Volume,Taker Buy Quote Volume,Ignore
0,2024-01-02,44179.55000000,45879.63000000,44148.34000000,44946.91000000,65146.40661000,2024-01-02 23:59:59.999,2944331821.82307620,2247532,33817.14447000,1527964124.23758480,0
1,2024-01-03,44946.91000000,45500.00000000,40750.00000000,42845.23000000,81194.55173000,2024-01-03 23:59:59.999,3507104563.74423210,2658041,39103.99162000,1687665202.95131440,0
2,2024-01-04,42845.23000000,44729.58000000,42613.77000000,44151.10000000,48038.06334000,2024-01-04 23:59:59.999,2095095359.49363480,1819944,23605.90059000,1030075267.83901300,0
3,2024-01-05,44151.10000000,44357.46000000,42450.00000000,44145.11000000,48075.25327000,2024-01-05 23:59:59.999,2100953924.36871550,2064845,24015.06426000,1049655019.14000940,0
4,2024-01-06,44145.12000000,44214.42000000,43397.05000000,43968.32000000,17835.06144000,2024-01-06 23:59:59.999,781212883.49160610,956642,9048.57073000,396341787.36387760,0
...,...,...,...,...,...,...,...,...,...,...,...,...
352,2024-12-19,100204.01000000,102800.11000000,95700.00000000,97461.86000000,55147.39800000,2024-12-19 23:59:59.999,5487546517.70775260,7273941,26898.63715000,2677636003.90650610,0
353,2024-12-20,97461.86000000,98233.00000000,92232.54000000,97805.44000000,62884.13570000,2024-12-20 23:59:59.999,6022845238.83232620,7429007,29463.23021000,2822939306.66307790,0
354,2024-12-21,97805.44000000,99540.61000000,96398.39000000,97291.99000000,23483.54143000,2024-12-21 23:59:59.999,2296232055.19592620,3717638,11726.94119000,1146732451.42181330,0
355,2024-12-22,97292.00000000,97448.08000000,94250.35000000,95186.27000000,19353.83036000,2024-12-22 23:59:59.999,1857819866.36039980,3633551,9293.39749000,892081877.76277080,0


In [10]:
# Guardar en CSV
df.to_csv("BTCUSDT_historical.csv", index=False)
print("Datos guardados en BTCUSDT_historical.csv")

Datos guardados en BTCUSDT_historical.csv


In [1]:
import yfinance as yf

# Definir el ticker de BTC en USD
btc_ticker = "BTC-USD"

# Descargar los datos históricos
btc_data = yf.download(tickers=btc_ticker, start="2010-01-01")

# Mostrar los primeros registros
print(btc_data)

# Guardar en un archivo CSV para análisis posterior
btc_data.to_csv("BTC_historical_data.csv")
print("Datos guardados en 'BTC_historical_data.csv'.")



[*********************100%***********************]  1 of 1 completed

Price              Close           High           Low           Open  \
Ticker           BTC-USD        BTC-USD       BTC-USD        BTC-USD   
Date                                                                   
2014-09-17    457.334015     468.174011    452.421997     465.864014   
2014-09-18    424.440002     456.859985    413.104004     456.859985   
2014-09-19    394.795990     427.834991    384.532013     424.102997   
2014-09-20    408.903992     423.295990    389.882996     394.673004   
2014-09-21    398.821014     412.425995    393.181000     408.084991   
...                  ...            ...           ...            ...   
2024-12-19  97490.953125  102748.148438  95587.679688  100070.687500   
2024-12-20  97755.929688   98098.914062  92175.179688   97484.695312   
2024-12-21  97224.726562   99507.101562  96426.523438   97756.195312   
2024-12-22  95104.937500   97360.265625  94202.187500   97218.320312   
2024-12-23  92920.437500   96336.734375  92509.835938   95092.57